In [1]:
DATA_NAME = 'coco-indoor' 
TRANSFORM = 'wavelet-vertical'
CHANNEL = 'green'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_green_vertical_wavelet_indoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,37748.37,0.71,0.20,0.13,0.00,0.00,-0.00,-0.00
3,0.71,8518.41,-0.09,0.05,-0.00,0.00,-0.00,0.00
4,0.20,-0.09,1589.61,0.01,0.00,-0.00,0.00,-0.00
5,0.13,0.05,0.01,263.61,0.00,0.00,0.00,-0.00
6,0.00,-0.00,0.00,0.00,37.98,0.00,0.00,0.00
7,0.00,0.00,-0.00,0.00,0.00,5.03,-0.00,0.00
8,-0.00,-0.00,0.00,0.00,0.00,-0.00,0.64,0.00
9,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,0.07


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,0.00004,0.00003,0.00004,0.00000,0.00000,-0.00000,-0.00001
3,0.00004,1.00000,-0.00002,0.00003,-0.00000,0.00001,-0.00003,0.00003
4,0.00003,-0.00002,1.00000,0.00002,0.00001,-0.00005,0.00004,-0.00002
5,0.00004,0.00003,0.00002,1.00000,0.00002,0.00002,0.00001,-0.00002
6,0.00000,-0.00000,0.00001,0.00002,1.00000,0.00002,0.00002,0.00002
7,0.00000,0.00001,-0.00005,0.00002,0.00002,1.00000,-0.00003,0.00001
8,-0.00000,-0.00003,0.00004,0.00001,0.00002,-0.00003,1.00000,0.00002
9,-0.00001,0.00003,-0.00002,-0.00002,0.00002,0.00001,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

1.0618473679425189

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.78317109e+08 8.88491920e+06 3.00848364e+05 7.94308093e+03
 1.54673683e+02 2.41082083e+00 2.94901129e-02 2.35629915e-34]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999408,-0.033874,-0.005964,-0.000985,-0.000142,-0.000019,-0.000002,-2.735336e-07
1,0.033658,0.998894,-0.032406,-0.005154,-0.000743,-0.000098,-0.000013,-1.272908e-06
2,0.007011,0.031989,0.998866,-0.034227,-0.004772,-0.000634,-0.000080,-8.981409e-06
3,0.001390,0.006176,0.033835,0.998715,-0.036892,-0.004756,-0.000605,-6.678458e-05
4,0.000250,0.001111,0.005952,0.036449,0.998290,-0.044954,-0.005591,-6.132658e-04
5,0.000044,0.000195,0.001048,0.006307,0.044288,0.996881,-0.064642,-6.924998e-03
6,0.000008,0.000037,0.000197,0.001194,0.008288,0.063181,0.991908,-1.097986e-01
7,0.000002,0.000008,0.000044,0.000265,0.001838,0.013897,0.109121,9.939295e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0005921944679898594,
 0.0011057007050210288,
 0.0011342098660616085,
 0.0012852584836428038,
 0.0017103760697416703,
 0.0031193103896908214,
 0.008091600503900898,
 0.006070460993557902]